<a href="https://colab.research.google.com/github/dheerajshah13/Human-Activity-Recognition/blob/main/multiheadConvLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from numpy import mean,std,dstack
from pandas import read_csv
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input,Dense,Flatten,Dropout,ConvLSTM2D,MaxPool3D
from keras.layers.merge import concatenate

In [ ]:
def load_file(filepath):
  dataframe = read_csv(filepath,header =None,delim_whitespace =True)
  return dataframe.values

In [ ]:
#for RAW data
def load_group(filenames, prefix=''):
  loaded = list()
  for name in filenames:
    data = load_file(prefix + name)
    loaded.append(data)
  loaded = dstack(loaded)
  return loaded


In [ ]:
def load_dataset_group(group, prefix=''):
  filepath = prefix + group +'/Inertial Signals/'
  filenames = list()
  filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt','total_acc_z_'+group+'.txt']
  filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt','body_acc_z_'+group+'.txt']
  filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt','body_gyro_z_'+group+'.txt']

  X = load_group(filenames,filepath)
  y = load_file(prefix + group + '/y_'+group+'.txt') 
  return X, y

In [ ]:
def load_dataset(prefix=''):
  trainX, trainy = load_dataset_group('train', prefix + '/content/drive/MyDrive/HAR/HARDataset/')
  testX, testy = load_dataset_group('test', prefix + '/content/drive/MyDrive/HAR/HARDataset/')
  trainy = trainy - 1
  testy = testy - 1
  trainy = to_categorical(trainy) 
  testy = to_categorical(testy) 
  return trainX, trainy, testX, testy


In [ ]:
def evaluate_model(trainX, trainy, testX, testy):
  verbose,epochs,batch_size = 0,25,64
  n_features, n_outputs = trainX.shape[2],trainy.shape[1]
  n_steps, n_length = 4,32
  trainX = trainX.reshape((trainX.shape[0],n_steps,1,n_length,n_features))
  testX = testX.reshape((testX.shape[0],n_steps,1,n_length,n_features))

  input1 = Input(shape=(n_steps,1,n_length,n_features))
  cvls1 = ConvLSTM2D(64,(1,3),activation='relu',return_sequences=True, padding='valid')(input1)
  drop1 = Dropout(0.5)(cvls1)
  pool1 = MaxPool3D(pool_size=(1,1,2))(drop1)
  flat1 = Flatten()(pool1)

  input2 = Input(shape=(n_steps,1,n_length,n_features))
  cvls2 = ConvLSTM2D(96,(1,5),activation='relu',return_sequences=True, padding='valid')(input2)
  drop2 = Dropout(0.5)(cvls2)
  pool2 = MaxPool3D(pool_size=(1,1,2))(drop2)
  flat2 = Flatten()(pool2)

  input3 = Input(shape=(n_steps,1,n_length,n_features))
  cvls3 = ConvLSTM2D(96,(1,7),activation='relu',return_sequences=True, padding='valid')(input3)
  drop3 = Dropout(0.5)(cvls3)
  pool3 = MaxPool3D(pool_size=(1,1,2))(drop3)
  flat3  = Flatten()(pool3)

  merged = concatenate([flat1,flat2,flat3])

  dense1 = Dense(100,activation ='relu')(merged)
  dense2 = Dense(64,activation = 'relu')(dense1)
  outputs = Dense(n_outputs,activation='softmax')(dense2)
  
  model = Model(inputs = [input1,input2,input3], outputs = outputs)
  plot_model(model,show_shapes=True, to_file='model_arch.png')
  model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
  model.fit([trainX,trainX,trainX], trainy, epochs=epochs, batch_size=batch_size,verbose=verbose)
  _, accuracy = model.evaluate([testX,testX,testX], testy, batch_size=batch_size, verbose=0)
  return accuracy



In [ ]:
def summarize_results(scores):
  print(scores)
  m, s = mean(scores), std(scores) 
  print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [ ]:
def run_experiment(repeats=10):
  trainX, trainy, testX, testy = load_dataset()
  scores = list()
  for r in range(repeats):
    score = evaluate_model(trainX, trainy, testX, testy) 
    score = score * 100.0
    print('>#%d: %.3f' % (r+1, score))
    scores.append(score)
  summarize_results(scores)

In [ ]:
run_experiment()

>#1: 91.822
>#2: 93.519
>#3: 92.026
>#4: 91.890
>#5: 92.094
>#6: 92.094
>#7: 91.992
>#8: 91.686
>#9: 90.804
>#10: 89.549
[91.82218909263611, 93.5188353061676, 92.02578663825989, 91.89005494117737, 92.09365248680115, 92.09365248680115, 91.99185371398926, 91.68646335601807, 90.80420732498169, 89.54869508743286]
Accuracy: 91.748% (+/-0.963)
